# Follow-up Detection Production Pipeline

This notebook implements an efficient follow-up detection algorithm for radiology reports using GPT-OSS:20b.

**Architecture:**
- Parallel classification with GPT-OSS:20b using ThreadPoolExecutor (16 workers)
- Results written back to `reports` with confidence scoring

**Performance:**
- Throughput: ~2.0 reports/second with 16 workers
- Estimated Runtime: ~5.8 days for 1M reports

**Columns Added:**
- `followup_detected` (boolean): True if follow-up recommended
- `followup_confidence` (string): "high" or "low"
- `followup_snippet` (string): Exact text indicating follow-up
- `followup_processed_at` (timestamp): Processing timestamp

## Cell 1: Imports & Configuration

In [ ]:
from pyspark.sql import SparkSession, functions as F, Window
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType
from datetime import datetime
from tqdm.notebook import tqdm
import requests
import json
import os

# Configuration
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://ollama.chatbot:11434")
MODEL = "gpt-oss:20b"
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY", "lake-writer")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")

if not S3_SECRET_KEY:
    from getpass import getpass
    S3_SECRET_KEY = getpass("Enter S3 secret key: ")

# Threading configuration
TOTAL_WORKERS = 16  # Match OLLAMA_NUM_PARALLEL for optimal throughput

# Processing configuration
BATCH_SIZE = 1000

# Priority filtering - process high-value reports first
# Reports must have BOTH:
# 1. Follow-up keywords (likely follow-ups)
# 2. Complex modalities (higher clinical value)
FOLLOWUP_KEYWORDS = [
    r"follow.?up", r"repeat", r"recommend", r"suggest", r"interval",
    r"re-?evaluate", r"re-?assess", r"correlat", r"clinical", r"short.?term",
    r"further", r"additional", r"consider", r"refer", r"return",
    r"continue", r"monitor", r"surveillance", r"re-?exam", r"comparison"
]
PRIORITY_MODALITIES = ["CT", "MR", "US", "MG", "PT"]  # PT = PET in your data

# Combined priority filter: (has keywords) AND (is priority modality)
PRIORITY_FILTER = (
    f"(lower(report_text) RLIKE '{('|'.join(FOLLOWUP_KEYWORDS))}') AND "
    f"(modality IN ({','.join(repr(m) for m in PRIORITY_MODALITIES)}))"
)

# Table names
SRC_TABLE = "default.reports"
DEST_TABLE = "default.reports"
ERROR_TABLE = "default.followup_errors"

print(f"⚙️  Configuration:")
print(f"  Ollama URL: {OLLAMA_URL}")
print(f"  Model: {MODEL}")
print(f"  Total workers: {TOTAL_WORKERS}")
print(f"  Batch size: {BATCH_SIZE:,}")
print(f"  Priority filter: Keywords AND Modality")
print(f"    - Must have follow-up keywords")
print(f"    - Must be in: {', '.join(PRIORITY_MODALITIES)}")

## Cell 2: Initialize Spark & Verify Connection

In [ ]:
# Start Spark session with optimizations
spark = SparkSession.builder.appName("followup-detection") \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore.hive:9083") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "800") \
    .enableHiveSupport().getOrCreate()

# Verify connection
try:
    spark.sql("SHOW DATABASES").show()
    print("✅ Spark connected to Delta Lake")
except Exception as e:
    print(f"❌ Error connecting to Delta Lake: {e}")
    raise

## Cell 5: Define Classification Functions

In [ ]:
PROMPT_TEMPLATE = """Does this report recommend follow-up for THIS patient?

Follow-up = patient-specific imaging/evaluation based on findings

YES if:
- Specific directive: "Repeat CT in 6 months for nodule"
- Referral for findings: "Clinical correlation recommended"
- Template applies: Patient has osteoporosis + template says "osteoporosis patients need follow-up"

NO if:
- Normal findings: "normal", "unremarkable", "stable"
- Template doesn't apply: Patient normal + template says "osteoporosis patients need follow-up"
- Generic advice: "all patients should take calcium"
- Report describes a follow-up exam, but does not request any future action

JSON: {{"follow_up": true/false, "confidence": "high"/"low", "snippet": "text"}}

If follow_up=false, snippet=""
Read FINDINGS first to check if templates apply

Report:
{report_text}"""

def classify_report(row):
    """Classify single report via Ollama API"""
    try:
        prompt = PROMPT_TEMPLATE.format(report_text=row.report_text)
        response = requests.post(
            f"{OLLAMA_URL}/api/generate",
            json={
                "model": MODEL,
                "prompt": prompt,
                "temperature": 0,
                "stream": False,
                "thinking": "low",
                "options": {
                    "num_predict": 2000,  # Prevent JSON truncation
                    "num_ctx": 4096
                }
            },
            timeout=120
        )
        
        if response.status_code != 200:
            raise Exception(f"HTTP {response.status_code}: {response.text}")
        
        resp_text = response.json()["response"].strip()

        # Try to parse JSON
        result = {
            "follow_up": False,
            "confidence": "low",
            "snippet": "",
        }
        try:
            obj = json.loads(resp_text)
        except json.JSONDecodeError:
            # Attempt to remove code block fencing that some models add
            if resp_text.startswith("```json\n"):
                resp_text = resp_text[len("```json\n"):]
            if resp_text.endswith("```"):
                resp_text = resp_text[:-len("```")]
            try:
                obj = json.loads(resp_text)
            except json.JSONDecodeError:
                # If still can't parse, return defaults with error
                raise Exception(f"Failed to parse JSON response: {resp_text}")
    
        # Update result with parsed values
        if isinstance(obj, dict):
            result["follow_up"] = bool(obj.get("follow_up", False))
            result["confidence"] = str(obj.get("confidence", "low")).lower()
            result["snippet"] = str(obj.get("snippet", ""))
    
        # Normalize confidence to "high" or "low"
        if result["confidence"] not in ["high", "low"]:
            result["confidence"] = "low"
        
        return {
            "message_control_id": row.message_control_id,
            "followup_detected": bool(result.get("follow_up", False)),
            "followup_confidence": result.get("confidence", "low"),
            "followup_snippet": str(result.get("snippet", "")),
            "error": None
        }
    except Exception as e:
        return {
            "message_control_id": row.message_control_id,
            "followup_detected": None,
            "followup_confidence": None,
            "followup_snippet": None,
            "error": str(e)
        }

print("✅ Classification function defined")

## TEST RUN: Process 100 Sample Reports

Run this cell to test the pipeline on a small sample before processing all reports.

In [ ]:
# TEST RUN: Process 100 sample reports
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

print("🧪 TEST RUN: Processing 100 sample reports...")
print("="*60)

# Load model into memory
print("\n📥 Loading model into memory...")
resp = requests.post(f"{OLLAMA_URL}/api/generate", json={"model": MODEL, "keep_alive": -1})
if resp.status_code == 200:
    print("✅ Model loaded into memory")
else:
    print(f"⚠️  Warning: Failed to load model: {resp.text}")

# Get 100 unprocessed reports for testing
test_count = (spark.table(DEST_TABLE)
             .filter(F.col("followup_processed_at").isNull())
             .count())

if test_count == 0:
    print("⚠️  No unprocessed reports found for testing!")
else:
    print(f"\n📊 Test sample: 100 reports (out of {test_count:,} unprocessed)")
    
    # Get 100 reports as Pandas DataFrame
    df_test = (spark.table(DEST_TABLE)
              .filter(F.col("followup_processed_at").isNull())
              .limit(100)
              .select("message_control_id", "report_text")
              .toPandas())
    
    # Time the processing
    start_time = time.time()
    
    # Process with simple ThreadPoolExecutor
    print("\n⏳ Processing with ThreadPoolExecutor...")
    results = []
    with ThreadPoolExecutor(max_workers=TOTAL_WORKERS) as executor:
        futures = {executor.submit(classify_report, row): idx 
                  for idx, row in df_test.iterrows()}
        
        for future in as_completed(futures):
            results.append(future.result())
    
    elapsed = time.time() - start_time
    
    # Convert results to DataFrame
    result_schema = StructType([
        StructField("message_control_id", StringType(), True),
        StructField("followup_detected", BooleanType(), True),
        StructField("followup_confidence", StringType(), True),
        StructField("followup_snippet", StringType(), True),
        StructField("error", StringType(), True)
    ])
    df_results = spark.createDataFrame(results, schema=result_schema)
    
    # Separate successes and failures
    df_success = df_results.filter(F.col("error").isNull()).drop("error")
    df_errors = df_results.filter(F.col("error").isNotNull())
    
    success_count = df_success.count()
    error_count = df_errors.count()
    total_processed = success_count + error_count
    
    print(f"\n✅ Test complete!")
    print(f"\n📈 Results:")
    print(f"  Processed: {success_count} reports")
    print(f"  Errors: {error_count} reports")
    print(f"  Time: {elapsed:.1f} seconds ({elapsed/max(1,total_processed):.2f}s per report)")
    print(f"  Throughput: {total_processed/max(1,elapsed):.2f} reports/second")
    
    # Show results breakdown
    if success_count > 0:
        print(f"\n📊 Follow-up Detection Breakdown:")
        df_success.groupBy("followup_detected", "followup_confidence").count().orderBy("followup_detected", "followup_confidence").show()
        
        # Show sample results
        print(f"\n🔍 Sample Positive Classifications:")
        df_success.filter(F.col("followup_detected") == True).select("message_control_id", "followup_confidence", "followup_snippet").limit(5).show(truncate=80)
        
        print(f"\n🔍 Sample Negative Classifications:")
        df_success.filter(F.col("followup_detected") == False).select("message_control_id", "followup_confidence", "followup_snippet").limit(5).show(truncate=80)
    
    # Show errors if any
    if error_count > 0:
        print(f"\n⚠️  Errors encountered:")
        df_errors.select("message_control_id", "error").limit(10).show(truncate=80)
    
    # Log errors to error table
    if error_count > 0:
        (df_errors
         .select(
             F.col("message_control_id"),
             F.col("error"),
             F.current_timestamp().alias("error_timestamp")
         )
         .write
         .format("delta")
         .mode("append")
         .saveAsTable(ERROR_TABLE))
        print(f"  ⚠️  {error_count} errors logged to {ERROR_TABLE}")
    
    # Write successful results to table
    if success_count > 0:
        df_success.createOrReplaceTempView("test_results")
        spark.sql(f"""
            MERGE INTO {DEST_TABLE} AS target
            USING test_results AS source
            ON target.message_control_id = source.message_control_id
            WHEN MATCHED THEN UPDATE SET
                target.followup_detected = source.followup_detected,
                target.followup_confidence = source.followup_confidence,
                target.followup_snippet = source.followup_snippet,
                target.followup_processed_at = current_timestamp()
        """)
        print(f"  ✅ {success_count} results written to {DEST_TABLE}")
    
    # Estimate full processing time
    total_unprocessed = spark.table(DEST_TABLE).filter(F.col("followup_processed_at").isNull()).count()
    if total_processed > 0:
        estimated_hours = (total_unprocessed * (elapsed / total_processed)) / 3600
        print(f"\n⏱️  Estimated time for {total_unprocessed:,} remaining reports: {estimated_hours:.1f} hours ({estimated_hours/24:.1f} days)")
        print(f"   Based on {elapsed/total_processed:.2f}s per report from this test run")

# Unload model from memory
#print("\n📤 Unloading model from memory...")
#requests.post(f"{OLLAMA_URL}/api/generate", json={"model": MODEL, "keep_alive": 0})

print("\n" + "="*60)
print("✅ Test run complete! Review results above before running full pipeline.")
print("="*60)

In [ ]:
from followup_review_dashboard import create_review_dashboard

# df_test is already a Pandas DataFrame with message_control_id and report_text
# Just merge it with the success results
df_review = df_success.toPandas().merge(
  df_test[["message_control_id", "report_text"]],
  on="message_control_id",
  how="left"
)

# Create the dashboard
create_review_dashboard(df_review, report_col='report_text')

## Cell 6: Process Reports with Checkpointing

In [ ]:
# Import parallel processing module
from parallel_processor import process_reports_in_batches

# Process all reports using simple ThreadPoolExecutor approach
# Priority filter processes keyword matches first
stats = process_reports_in_batches(
    spark=spark,
    classify_func=classify_report,
    dest_table=DEST_TABLE,
    error_table=ERROR_TABLE,
    ollama_url=OLLAMA_URL,
    model=MODEL,
    total_workers=TOTAL_WORKERS,
    batch_size=BATCH_SIZE,
    priority_filter=PRIORITY_FILTER  # Process keyword matches first
)

print("\n✅ Processing complete!")
print(f"\n📊 Final Statistics:")
print(f"  Total processed: {stats['total_processed']:,}")
print(f"  Total errors: {stats['total_errors']:,}")
print(f"  Total time: {stats['elapsed_time']/3600:.2f} hours")

## Cell 7: Validation & Summary

In [ ]:
# Summary statistics
df_final = spark.table(DEST_TABLE)

print("\n" + "="*60)
print("📊 FOLLOW-UP DETECTION SUMMARY")
print("="*60)

print("\nResults by follow-up status:")
df_final.groupBy("followup_detected").count().orderBy("followup_detected").show()

print("\nResults by follow-up status and confidence:")
df_final.groupBy("followup_detected", "followup_confidence").count().orderBy("followup_detected", "followup_confidence").show()

print("\nProcessing statistics:")
processed = df_final.filter(F.col("followup_processed_at").isNotNull()).count()
total_reports = df_final.count()
print(f"  Total reports: {total_reports:,}")
print(f"  Processed: {processed:,} ({100*processed/total_reports:.1f}%)")
print(f"  Unprocessed: {total_reports - processed:,}")

followup_count = df_final.filter(F.col("followup_detected") == True).count()
if processed > 0:
    print(f"  Follow-up rate: {100*followup_count/processed:.1f}%")

followup_high_conf = df_final.filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "high")).count()
if followup_count > 0:
    print(f"  High confidence follow-ups: {followup_high_conf:,} ({100*followup_high_conf/followup_count:.1f}%)")

print("\n🔍 Sample High Confidence Follow-up Reports:")
(df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "high"))
    .select("message_control_id", "followup_confidence", "followup_snippet")
    .limit(10)
    .show(truncate=80)
)

print("\n🔍 Sample Low Confidence Follow-up Reports:")
(df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "low"))
    .select("message_control_id", "followup_confidence", "followup_snippet")
    .limit(10)
    .show(truncate=80)
)

# Check errors
df_errors = spark.table(ERROR_TABLE)
error_count = df_errors.count()
print(f"\n⚠️  Error Statistics:")
print(f"  Total errors: {error_count:,}")
if error_count > 0:
    print(f"  Error rate: {100*error_count/(processed+error_count):.2f}%")
    print("\n  Top error types:")
    df_errors.groupBy("error").count().orderBy(F.col("count").desc()).limit(5).show(truncate=80)

# Export samples for validation
print("\n💾 Exporting validation samples...")
output_dir = "/home/jovyan/followup_validation"
os.makedirs(output_dir, exist_ok=True)

# High confidence positive samples
high_conf_sample = (df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "high"))
    .select("message_control_id", "followup_detected", "followup_confidence", "followup_snippet", "report_text")
    .limit(100)
    .toPandas())
high_conf_sample.to_csv(f"{output_dir}/followup_high_confidence_sample.csv", index=False)
print(f"  ✅ Saved: {output_dir}/followup_high_confidence_sample.csv")

# Low confidence positive samples
low_conf_sample = (df_final
    .filter((F.col("followup_detected") == True) & (F.col("followup_confidence") == "low"))
    .select("message_control_id", "followup_detected", "followup_confidence", "followup_snippet", "report_text")
    .limit(100)
    .toPandas())
low_conf_sample.to_csv(f"{output_dir}/followup_low_confidence_sample.csv", index=False)
print(f"  ✅ Saved: {output_dir}/followup_low_confidence_sample.csv")

# Negative samples
negative_sample = (df_final
    .filter(F.col("followup_detected") == False)
    .select("message_control_id", "followup_detected", "followup_confidence", "followup_snippet", "report_text")
    .limit(100)
    .toPandas())
negative_sample.to_csv(f"{output_dir}/followup_negative_sample.csv", index=False)
print(f"  ✅ Saved: {output_dir}/followup_negative_sample.csv")

print("\n" + "="*60)
print("🎉 PROCESSING COMPLETE")
print("="*60)

# Interactive Dashboard Review
print("\n" + "="*60)
print("📱 INTERACTIVE DASHBOARD REVIEW")
print("="*60)

from followup_review_dashboard import create_review_dashboard

# Stratified sample across modalities AND classifications
df_processed = df_final.filter(F.col("followup_processed_at").isNotNull())

print("\n🔄 Creating stratified sample by modality...")
# Get 10 samples per modality per classification type
df_review = (df_processed
    .filter(F.col("modality").isNotNull())
    .withColumn("category", F.concat_ws("_", 
                                        F.col("modality"),
                                        F.col("followup_detected").cast("string"),
                                        F.coalesce(F.col("followup_confidence"), F.lit("null"))))
    .withColumn("row_num", F.row_number().over(
        Window.partitionBy("category").orderBy(F.rand())
    ))
    .filter(F.col("row_num") <= 10)  # 10 per category
    .select("message_control_id", "modality", "followup_detected", 
            "followup_confidence", "followup_snippet", "report_text")
    .toPandas())

print(f"\n📊 Loaded {len(df_review)} reports stratified by modality")
print("\nSample distribution:")
print(df_review.groupby(['modality', 'followup_detected', 'followup_confidence']).size())

# Launch dashboard
print("\n🚀 Launching interactive dashboard...")
create_review_dashboard(df_review, report_col='report_text')